# Inspecting a LAS/LAZ file

In [ ]:
pdal --version

In [ ]:
ls -l ./data

In general, it is best to use absolute paths to specify paths to datafiles.  Relative paths may cause issues.

In [ ]:
pwd

In [ ]:
laz_file=$PWD'/data/FoxIsland.laz'
CWD=`pwd`
echo $laz_file
echo $CWD

## Metadata

In [ ]:
pdal info --metadata $laz_file

To get the schema for a dataset:

In [ ]:
pdal info --schema ./data/FoxIsland.laz

To get an overall metadata summary for a dataset:

In [ ]:
pdal info $laz_file --summary

## Printing Points

Print the values for the first point in the file:

In [ ]:
pdal info $laz_file -p 0

Print the values for the first 5 points in the file

In [ ]:
pdal info $laz_file -p 0-10

Print the first 10 points, and extract only the elevation values using bash commands

In [ ]:
pdal info $laz_file -p 0-10|grep  "Z"|awk '{print $2}' FS=': '

jq is a command line JSON parser, and can be a useful tool for these types of operations. Note if returning more than one point, specify "[]" to return all the points in the array.

In [ ]:
pdal info $laz_file -p 0-10|jq -r .points.point[].Z

## Attribute Statistics

Use the [info application](https://pdal.io/en/2.4.3/apps/info.html) with the --stats flag and perform filtering to get a summary of the classifications for a given lidar file.

In [ ]:
pdal info ./data/FoxIsland.laz --stats --filters.stats.dimensions=Classification  --filters.stats.count=Classification

Perform both a metadata dump and count of Classification and ReturnNumber and output to a JSON file:

In [ ]:
pdal info --metadata --stats --filters.stats.dimensions=ReturnNumber,Classification --filters.stats.count=ReturnNumber,Classification ./data/FoxIsland.laz > ./data/fileinfo.json

## Reprojection

In [ ]:
ls

Run a pipeline to project a file from Oregon Lambert to Geographic Lat/Lon coordindates:

In [ ]:
pdal pipeline ./pipelines/Reproject_Ex.json

In [ ]:
ls ./data

Inspect the coordinates to make sure the xy values make sense..

In [ ]:
pdal info --metadata ./data/FoxIsland_4326.laz

# Tiling data

In [59]:
pdal pipeline ./pipelines/TileLAZ.json

# Thinning data

Point cloud files can often be quite large and cumbersome to work with.  Depending on the objective, it is often useful to thin a dataset in order to make it easier and faster to work with. The [filters.sample](https://pdal.io/en/2.5.3/stages/filters.sample.html#filters-sample) utilizes a Poisson sampling to thin the dataset.

In [60]:
pdal translate ./data/FoxIsland.laz ./data/FoxIsland_Thin1m.laz sample --filters.sample.radius=1

## Calculating a Boundary

Utilizing the info command, the boundary of a dataset can be obtained by using the "--boundary" flag.  This will output the boundary in WKT format in JSON-formatted output.

In [ ]:
ls

Use the "--boundary" flag to get the boundary of data over the Siuslaw River:

In [61]:
pdal info ./data/Siuslaw.laz --boundary 

{
  "boundary":
  {
    "area": 121120450.7,
    "avg_pt_per_sq_unit": 36.11257303,
    "avg_pt_spacing": 3.432894977,
    "boundary": "POLYGON ((418069.16 839889.38,420190.35 839889.38,420367.12 842032.55,418245.93 843257.22,417538.86 844481.89,417715.63 845400.39,419660.05 845706.56,421958.01 844788.06,422841.84 845094.22,426200.39 842951.05,429735.71 842951.05,430442.77 843563.39,431326.6 843257.22,432563.96 844788.06,431856.9 845400.39,432033.66 846318.89,431326.6 846931.23,431326.6 848155.9,432563.96 849074.4,433978.09 848462.07,434508.38 849992.9,433094.26 850605.24,432387.19 851829.91,431503.37 851523.74,428851.88 853054.58,428675.11 853973.08,427614.52 854585.42,426730.69 854279.25,425139.79 855197.75,423902.43 857340.92,422488.31 857340.92,422311.54 855810.09,423548.9 853666.91,426553.92 852136.08,426730.69 851217.57,427614.52 850911.41,428851.88 848768.23,428851.88 846931.23,428144.81 845706.56,427260.98 845400.39,423372.14 847849.73,418776.22 847849.73,417715.63 848462.07,41

However, to get a boundary in vector format to visualize in a GIS or Google Earth requires some additional steps.  The PDAL command, [tindex](https://pdal.io/en/2.5.3/apps/tindex.html#tindex-command) is used to create a boundary that utilizes the [hexbin filter](https://pdal.io/en/2.5.3/stages/filters.hexbin.html#filters-hexbin)

To get a basic boundary:

In [62]:
pdal tindex create --tindex ./data/Siuslaw_bounds.shp --filespec ./data/Siuslaw.laz -f "ESRI Shapefile"

Warning 6: Field modified create as date field, though DateTime requested.
Warning 6: Field created create as date field, though DateTime requested.
Warning 1: Value 'PROJCS["NAD83 / Oregon GIC Lambert (ft)",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",41.75],PARAMETER["central_meridian",-120.5],PARAMETER["standard_parallel_1",43],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",1312335.958],PARAMETER["false_northing",0],UNIT["foot",0.3048,AUTHORITY["EPSG","9002"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","2992"]]' of field srs has been truncated to 254 characters.  This warning will not be emitted any more for that layer.


Load the shapefile into a GIS to see its extent:
![Siuslaw Bounds](./images/SiuslawRiverBounds.jpeg)

For rough estimations of boundaries, this is usually sufficient. To obtain a more precise fit of the data alter some of the parameters in the [filters.hexbin](https://pdal.io/en/2.5.3/stages/filters.hexbin.html#filters-hexbin) command.  The "edge_size" parameter is particularly useful for this scenario as it controls the size of the hexagon boundaries used to estimate whether a section of the dataset should be considered. Finding an appropriate value for edge_size can be an iterative process.  For example, try using a vlue of "50" units (for this dataset, the units are feet).

In [63]:
pdal tindex create --tindex ./data/Siuslaw_bounds50.shp --filters.hexbin.edge_size=50 --filespec ./data/Siuslaw.laz -f "ESRI Shapefile"

Warning 6: Field modified create as date field, though DateTime requested.
Warning 6: Field created create as date field, though DateTime requested.
Warning 1: Value 'PROJCS["NAD83 / Oregon GIC Lambert (ft)",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",41.75],PARAMETER["central_meridian",-120.5],PARAMETER["standard_parallel_1",43],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",1312335.958],PARAMETER["false_northing",0],UNIT["foot",0.3048,AUTHORITY["EPSG","9002"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","2992"]]' of field srs has been truncated to 254 characters.  This warning will not be emitted any more for that layer.


Load this version of the boundary into a GIS:
![Siuslaw Bounds_Fit](./images/SiuslawRiverBounds_Fit.png)

Note how this is a much better fit to the data, and shows regions where there is no data (over the water). However, use caution with the edge_size parameter as setting it too low might not capture an appropriate amount of data, and mis-represent the data coverage.

### Visualizing Point Density

In [64]:
pdal density ./data/Siuslaw.laz -o ./data/Siuslaw_density.shp -f "ESRI Shapefile"

Size of the hexagons can be controlled with the optional parameter, "--filters.hexbin.edge_size"

In [65]:
pdal density --filters.hexbin.edge_size=50 ./data/Siuslaw.laz -o ./data/Siuslaw_density50.shp -f "ESRI Shapefile"

## Filtering Noise

In [66]:
pdal pipeline ./pipelines/NoiseFilter.json

In [67]:
pdal info ./data/FoxIsland_Clean.laz --stats --filters.stats.dimensions=Classification  --filters.stats.count=Classification

{
  "file_size": 17452659,
  "filename": "./data/FoxIsland_Clean.laz",
  "now": "2023-04-29T16:38:01-0600",
  "pdal_version": "2.5.3 (git-version: Release)",
  "reader": "readers.las",
  "stats":
  {
    "statistic":
    [
      {
        "average": 1.158417543,
        "count": 2996783,
        "counts":
        [
          "1.000000/2522040",
          "2.000000/474743"
        ],
        "maximum": 2,
        "minimum": 1,
        "name": "Classification",
        "position": 0,
        "stddev": 0.3651321262,
        "variance": 0.1333214696
      }
    ]
  }
}


# Cropping Data

[filters.crop](https://pdal.io/en/2.5.3/stages/filters.crop.html) removes points that fall outside or inside a cropping bounding box. The “polygon” option takes a WKT-formatted string to apply the clipping mask.

In [68]:
pdal pipeline ./pipelines/Clip_ex.json

# Ground Classifications

In [69]:
pdal pipeline ./pipelines/ExtractGround.json

In [70]:
pdal pipeline ./pipelines/CreateGround.json

# Create Rasters

In [77]:
pdal pipeline ./pipelines/CreateDSM.json

In [72]:
pdal pipeline ./pipelines/CreateDTM.json

Use GDAL raster calculator to difference the DSM and DTM to create a CHM

In [74]:
 gdal_calc.py -A ./data/FoxIsland_DSM.tif -B ./data/FoxIsland_DTM.tif --outfile ./data/CHM.tif --calc="A-B" --NoDataValue=-9999 --extent intersect



0.. 25.. 50.. 75.. 100 - Done


# GDAL Visualizations
- The [gdaldem](https://gdal.org/programs/gdaldem.html) application is a quick and easy way to visualize raster products such as
   - Hillshade
   - Slope
   - Aspect
   - Roughness

In [80]:
pdal pipeline ./pipelines/DevilsTower.json

In [81]:
gdaldem hillshade ./data/DevilsTower_Ground.tif ./data/DevilsTower_Ground_HS.tif -z 1 -az 315 -alt 45 

0...10...20...30...40...50...60...70...80...90...100 - done.


Experiment with some of the different parameters.  For example, try out the "multidirectional" option.  Multidirectional shading is a combination of hillshading illuminated from 225 deg, 270 deg, 315 deg, and 360 deg azimuth.

In [82]:
gdaldem hillshade ./data/DevilsTower_Ground.tif ./data/DevilsTower_Ground_HSMulti.tif -z 1 -multidirectional

0...10...20...30...40...50...60...70...80...90...100 - done.
